In [1]:
from datetime import datetime, timedelta
from pandas import DataFrame, concat, date_range, ExcelWriter, read_csv, concat
# from pandas import DataFrame, ExcelWriter, date_range, to_datetime
import os
from numpy import isnan
import time
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

parent = os.path.dirname(os.path.abspath("__file__"))
record_path = os.path.join(parent, "FollowBroker")
if not os.path.isdir(record_path):
    os.makedirs(record_path)

In [2]:
def readDailyRecord(date:datetime):
    ob_f_path = os.path.join(record_path, f'{date.strftime("%Y%m%d")}_OverBuy.csv')
    os_f_path = os.path.join(record_path, f'{date.strftime("%Y%m%d")}_OverSell.csv')
    ob_df, os_df = DataFrame(), DataFrame()
    if os.path.isfile(ob_f_path):
        try:
            ob_df = read_csv(ob_f_path, dtype=str)
            ob_df.columns = ['Ticker','Name', 'Qty', 'Value','Price','Volume', 'Ratio']
        except:
            pass
        try:
            os_df = read_csv(os_f_path, dtype=str)
            os_df.columns = ['Ticker','Name', 'Qty', 'Value','Price','Volume', 'Ratio']
        except:
            pass
    return ob_df, os_df

In [3]:
def changeType(x):
    if isinstance(x, float) or isinstance(x, int):
        if isnan(x):
            return x
#         else:
#             return x
    elif isinstance(x, str):
        return float(x.replace(',',''))

# First Time Aggregate

In [4]:
ob_dfs = {}
os_dfs = {}
for date in date_range(datetime(2022,5,9), datetime.today()):
    ob_df, os_df = readDailyRecord(date)
    if ob_df.empty and os_df.empty:
        continue
    ob_dfs[date] = ob_df
    os_dfs[date] = os_df

In [5]:
keys = sorted(set(ob_dfs.keys()).intersection(os_dfs.keys()))

In [6]:
tickers_ob = concat(list(ob_dfs.values())).Ticker.unique()
tickers_os = concat(list(os_dfs.values())).Ticker.unique()
tickers = list(set(tickers_ob).intersection(tickers_os))

In [7]:
ob_dfs[date]

KeyError: Timestamp('2022-06-21 00:00:00', freq='D')

In [8]:
results = {}

for date in date_range(datetime(2022,5,9), datetime.today()):
    if date in ob_dfs.keys():
        tmp_ob = list(ob_dfs[date].T.to_dict().values())
        for ob_ in tmp_ob:
            results[ob_['Ticker']] = results.get(ob_['Ticker'],[])
            results[ob_['Ticker']].append({
                "交易日期":date.strftime("%Y-%m-%d"),
                '代號':ob_['Ticker'],
                '名稱':ob_['Name'],
                '買均':ob_['Price'],
                '買量':ob_['Qty'],
                '賣均':float('nan'),
                '賣量':float('nan')
            })
    if date in os_dfs.keys():
        tmp_os = list(os_dfs[date].T.to_dict().values())
        for os_ in tmp_os:
            results[os_['Ticker']] = results.get(os_['Ticker'], [])
            results[os_['Ticker']].append({
                "交易日期":date.strftime("%Y-%m-%d"),
                '代號':os_['Ticker'],
                '名稱':os_['Name'],
                '買均':float('nan'),
                '買量':float('nan'),
                '賣均':os_['Price'],
                '賣量':os_['Qty']
            })

In [50]:
results.keys()

dict_keys(["'2353", "'3374", "'3596", "'6470", "'00664R", "'3231", "'6235", "'2618", "'1815", "'2880", "'3710", "'9962", "'6111", "'2610", "'2406", "'1605", "'2317", "'1795", "'3481", "'2303", "'5351", "'00637L", "'2371", "'2883B", "'00632R", "'3035", "'3552", "'1102", "'2884", "'3037", "'8069", "'3532", "'3520", "'4433", "'5258", "'6163", "'00891", "'2405", "'2614", "'2834", "'4906", "'5272", "'1560", "'2915", "'3558", "'6605", "'2887", "'2888", "'3645", "'4721", "'6217", "'00671R", "'00878", "'2603", "'2609", "'4908", "'6026", "'2027", "'2515", "'6546", "'1524", "'2385", "'4931", "'2301", "'2634", "'00669R", "'00893", "'6226", "'8155"])

In [70]:
def DoSummary(ticker, results):
    tmp_df = DataFrame(results[ticker]).copy(deep=True)
    tmp_df['買均'] = tmp_df['買均'].apply(lambda x: changeType(x))
    tmp_df['買量'] = tmp_df['買量'].apply(lambda x: changeType(x))
    tmp_df['賣均'] = tmp_df['賣均'].apply(lambda x: changeType(x))
    tmp_df['賣量'] = tmp_df['賣量'].apply(lambda x: changeType(x))
    tmp_df['累計買均'] = float('nan')
    tmp_df['累計買量'] = tmp_df['買量'].fillna(0).cumsum() + tmp_df['賣量'].fillna(0).cumsum()
    tmp_df['損益'] = float('nan')
    tmp_df['進場手續費'] = float('nan')
    tmp_df['出場手續費'] = float('nan')
    tmp_df['交易稅'] = float('nan')
    tmp_df['淨損益'] = float('nan')
    tmp_df['餘量'] = tmp_df['買量']
    tmp_df['餘累買量'] = tmp_df['累計買量']
    display(tmp_df)

    for row in tmp_df.itertuples():
        if not isnan(row.賣均) and isnan(row.損益) and row.Index: # 有賣出
            
            exit_qty = abs(row.賣量)
            exit_price = row.賣均
            pnl = 0
            exit_commission = entry_commission = 0
            tax = 0
#             print(row, isnan(row.賣均), isnan(row.損益), exit_qty, exit_qty > 0)
#             print(range(tmp_df.shape[0], row.Index))
            for i in range(0, min(tmp_df.shape[0], row.Index)):
#                 print(i, exit_qty)
                if tmp_df.loc[i, '餘量'] <= 0: continue
#                 print(exit_qty)
                if exit_qty > 0:
#                     display(tmp_df.head(i+1))
                    if tmp_df.loc[i, '餘量']:
                        covered_qty = min(tmp_df.loc[i, '餘量'] if not isnan(tmp_df.loc[i, '餘量']) else tmp_df.loc[i, '買量'], exit_qty)
#                         print(covered_qty, isnan(covered_qty))
                        if isnan(covered_qty):
                            covered_qty = tmp_df.loc[i, '餘量'] = 0
                            continue
                        entry_price = tmp_df.loc[i, '買均']
#                         print(tmp_df.loc[i])
#                         print(row.Index,i, entry_price, exit_price, exit_qty, covered_qty)
                        exit_qty -= covered_qty
                        pnl += int((exit_price - entry_price) * 1000 * covered_qty)
                        entry_commission += int(round(entry_price * 1.425e-3 * .3 * covered_qty * 1000))
                        exit_commission += int(round(exit_price * 1.425e-3 * .3 * covered_qty * 1000))
                        tax += int(round(exit_price * 3e-3 * covered_qty * 1000))
                        if exit_qty > 0:
                            tmp_df.loc[i, '餘量'] = 0
                        else:
                            tmp_df.loc[i, '餘量'] = abs(row.賣量) - covered_qty# tmp_df.loc[i, '餘量']
            tmp_df.loc[row.Index, '損益'] = int(pnl)
            tmp_df.loc[row.Index, '進場手續費'] = int(entry_commission)
            tmp_df.loc[row.Index, '出場手續費'] = int(exit_commission)
            tmp_df.loc[row.Index, '交易稅'] = int(tax)
            tmp_df.loc[row.Index, '淨損益'] = int(pnl - (entry_commission + exit_commission + tax))

            tmp_df['餘累買量'] = tmp_df['餘量'].fillna(0).cumsum()
            total_qty = 0
            # 累計買均
            tmp_qty = row.買量 if not isnan(row.買量) else 0
            tmp_price = row.買均 if not isnan(row.買均) else 0
            if not tmp_qty and not tmp_price and row.Index:
                tmp_df.loc[row.Index, '累計買均'] = tmp_df.loc[row.Index-1, '累計買均']
        else:
            tmp_qty = row.買量 if not isnan(row.買量) else 0
            tmp_price = row.買均 if not isnan(row.買均) else 0
            if not row.Index:
                tmp_df.loc[row.Index, '累計買均'] = tmp_price * tmp_qty / row.累計買量
            else:
                tmp_df.loc[row.Index, '累計買均'] = (tmp_price * tmp_qty + tmp_df.loc[row.Index-1, '累計買均'] * tmp_df.loc[row.Index-1, '累計買量'] )/ row.累計買量

    tmp_df['餘買均'] = 0
    for row in tmp_df.itertuples():
        tmp_qty = row.餘量 if not isnan(row.餘量) else 0
        tmp_price = row.買均 if not isnan(row.買均) else 0
        if not row.Index:
            tmp_df.loc[row.Index, '餘買均'] = tmp_price * tmp_qty / row.餘累買量 if row.餘累買量 and not isnan(row.餘累買量) else 0
        else:
            tmp_df.loc[row.Index, '餘買均'] = (tmp_price * tmp_qty + tmp_df.loc[row.Index-1, '餘買均'] * tmp_df.loc[row.Index-1, '餘累買量'] )/ row.餘累買量


    tmp_ticker = ticker.replace("\'","")
    display(tmp_df)
    tmp_df.to_csv(os.path.join(record_path, f'{tmp_ticker}_Summary.csv'), index=False, encoding='utf-8-sig')

In [71]:
for ticker in results.keys():
    print(ticker)
# ticker = '\'3374'
    DoSummary(ticker, results)

'2353


,交易日期,代號,名稱,買均,買量,賣均,賣量,累計買均,累計買量,損益,進場手續費,出場手續費,交易稅,淨損益,餘量,餘累買量
0,2022-05-09,'2353,宏��,27.53,274.0,NaN,NaN,NaN,274.0,NaN,NaN,NaN,NaN,NaN,274.0,274.0
1,2022-05-10,'2353,宏��,27.82,781.0,NaN,NaN,NaN,1055.0,NaN,NaN,NaN,NaN,NaN,781.0,1055.0
2,2022-05-11,'2353,宏��,27.83,933.0,NaN,NaN,NaN,1988.0,NaN,NaN,NaN,NaN,NaN,933.0,1988.0
3,2022-05-12,'2353,宏��,27.69,604.0,NaN,NaN,NaN,2592.0,NaN,NaN,NaN,NaN,NaN,604.0,2592.0
4,2022-05-13,'2353,宏��,28.30,1221.0,NaN,NaN,NaN,3813.0,NaN,NaN,NaN,NaN,NaN,1221.0,3813.0
5,2022-05-16,'2353,宏��,28.58,1448.0,NaN,NaN,NaN,5261.0,NaN,NaN,NaN,NaN,NaN,1448.0,5261.0
6,2022-05-17,'2353,宏��,28.44,267.0,NaN,NaN,NaN,5528.0,NaN,NaN,NaN,NaN,NaN,267.0,5528.0
7,2022-05-19,'2353,宏��,28.18,446.0,NaN,NaN,NaN,5974.0,NaN,NaN,NaN,NaN,NaN,446.0,5974.0
8,2022-05-31,'2353,宏��,NaN,NaN,29.64,-781.0,NaN,5193.0,NaN,NaN,NaN,NaN,NaN,NaN,5193.0
9,2022-06-01,'2353,宏��,29.17,463.0,NaN,NaN,NaN,5656.0,NaN,NaN,NaN,NaN,NaN,463.0,5656.0


,交易日期,代號,名稱,買均,買量,賣均,賣量,累計買均,累計買量,損益,進場手續費,出場手續費,交易稅,淨損益,餘量,餘累買量,餘買均
0,2022-05-09,'2353,宏��,27.53,274.0,NaN,NaN,27.530000,274.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.000000
1,2022-05-10,'2353,宏��,27.82,781.0,NaN,NaN,27.744682,1055.0,NaN,NaN,NaN,NaN,NaN,274.0,274.0,27.820000
2,2022-05-11,'2353,宏��,27.83,933.0,NaN,NaN,27.784723,1988.0,NaN,NaN,NaN,NaN,NaN,933.0,1207.0,27.827730
3,2022-05-12,'2353,宏��,27.69,604.0,NaN,NaN,27.762650,2592.0,NaN,NaN,NaN,NaN,NaN,604.0,1811.0,27.781795
4,2022-05-13,'2353,宏��,28.30,1221.0,NaN,NaN,27.934721,3813.0,NaN,NaN,NaN,NaN,NaN,1221.0,3032.0,27.990478
5,2022-05-16,'2353,宏��,28.58,1448.0,NaN,NaN,28.112323,5261.0,NaN,NaN,NaN,NaN,NaN,1448.0,4480.0,28.181020
6,2022-05-17,'2353,宏��,28.44,267.0,NaN,NaN,28.128149,5528.0,NaN,NaN,NaN,NaN,NaN,267.0,4747.0,28.195587
7,2022-05-19,'2353,宏��,28.18,446.0,NaN,NaN,28.132020,5974.0,NaN,NaN,NaN,NaN,NaN,446.0,5193.0,28.194248
8,2022-05-31,'2353,宏��,NaN,NaN,29.64,-781.0,28.132020,5193.0,1500879.0,9255.0,9896.0,69446.0,1412282.0,NaN,5193.0,28.194248
9,2022-06-01,'2353,宏��,29.17,463.0,NaN,NaN,28.216989,5656.0,NaN,NaN,NaN,NaN,NaN,463.0,5656.0,28.274123


'3374


,交易日期,代號,名稱,買均,買量,賣均,賣量,累計買均,累計買量,損益,進場手續費,出場手續費,交易稅,淨損益,餘量,餘累買量
0,2022-05-09,'3374,精材,132.05,321.0,NaN,NaN,NaN,321.0,NaN,NaN,NaN,NaN,NaN,321.0,321.0
1,2022-05-11,'3374,精材,NaN,NaN,135.86,-337.0,NaN,-16.0,NaN,NaN,NaN,NaN,NaN,NaN,-16.0
2,2022-05-17,'3374,精材,139.99,299.0,NaN,NaN,NaN,283.0,NaN,NaN,NaN,NaN,NaN,299.0,283.0
3,2022-05-18,'3374,精材,NaN,NaN,141.94,-154.0,NaN,129.0,NaN,NaN,NaN,NaN,NaN,NaN,129.0
4,2022-05-19,'3374,精材,NaN,NaN,141.82,-152.0,NaN,-23.0,NaN,NaN,NaN,NaN,NaN,NaN,-23.0


...

,交易日期,代號,名稱,買均,買量,賣均,賣量,累計買均,累計買量,損益,進場手續費,出場手續費,交易稅,淨損益,餘量,餘累買量
0,2022-06-20,'6226,光鼎,NaN,NaN,14.25,-103.0,NaN,-103.0,NaN,NaN,NaN,NaN,NaN,NaN,-103.0


,交易日期,代號,名稱,買均,買量,賣均,賣量,累計買均,累計買量,損益,進場手續費,出場手續費,交易稅,淨損益,餘量,餘累買量,餘買均
0,2022-06-20,'6226,光鼎,NaN,NaN,14.25,-103.0,-0.0,-103.0,NaN,NaN,NaN,NaN,NaN,NaN,-103.0,-0.0


'8155


,交易日期,代號,名稱,買均,買量,賣均,賣量,累計買均,累計買量,損益,進場手續費,出場手續費,交易稅,淨損益,餘量,餘累買量
0,2022-06-20,'8155,博智,NaN,NaN,130.0,-226.0,NaN,-226.0,NaN,NaN,NaN,NaN,NaN,NaN,-226.0


,交易日期,代號,名稱,買均,買量,賣均,賣量,累計買均,累計買量,損益,進場手續費,出場手續費,交易稅,淨損益,餘量,餘累買量,餘買均
0,2022-06-20,'8155,博智,NaN,NaN,130.0,-226.0,-0.0,-226.0,NaN,NaN,NaN,NaN,NaN,NaN,-226.0,-0.0


# Version 1

In [72]:
for i, key in enumerate(keys):
    print(key)
    try:
        tmp_ob = ob_dfs[key].copy(deep=True)
        tmp_os = os_dfs[key].copy(deep=True)

        if not tmp_ob.empty:
            tmp_ob['Date'] = date
            for row in tmp_ob.itertuples():
                if not changeType(row.Qty):continue
                Agg_df.loc[key, str(row.Ticker).replace('\'','') + "_" + row.Name] = changeType(row.Qty)
        if not tmp_os.empty:
            tmp_os['Date'] = date
            for row in tmp_os.itertuples():
                if not changeType(row.Qty):continue
                Agg_df.loc[key, str(row.Ticker).replace('\'','') + "_" + row.Name] = changeType(row.Qty)
    except Exception as e:
        print(e, row)
        break

2021-01-04 00:00:00
2021-01-05 00:00:00
2021-01-06 00:00:00
2021-01-07 00:00:00
2021-01-08 00:00:00
2021-01-11 00:00:00
2021-01-12 00:00:00
2021-01-13 00:00:00
2021-01-14 00:00:00
2021-01-15 00:00:00
2021-01-18 00:00:00
2021-01-19 00:00:00
2021-01-20 00:00:00
2021-01-21 00:00:00
2021-01-22 00:00:00
2021-01-25 00:00:00
2021-01-26 00:00:00
2021-01-27 00:00:00
2021-01-28 00:00:00
2021-01-29 00:00:00
2021-02-01 00:00:00
2021-02-02 00:00:00
2021-02-03 00:00:00
2021-02-04 00:00:00
2021-02-05 00:00:00
2021-02-17 00:00:00
2021-02-18 00:00:00
2021-02-19 00:00:00
2021-02-22 00:00:00
2021-02-23 00:00:00
2021-02-24 00:00:00
2021-02-25 00:00:00
2021-02-26 00:00:00
2021-03-02 00:00:00
2021-03-03 00:00:00
2021-03-04 00:00:00
2021-03-05 00:00:00
2021-03-08 00:00:00
2021-03-09 00:00:00
2021-03-10 00:00:00
2021-03-11 00:00:00
2021-03-12 00:00:00
2021-03-15 00:00:00
2021-03-16 00:00:00
2021-03-17 00:00:00
2021-03-18 00:00:00
2021-03-19 00:00:00
2021-03-22 00:00:00
2021-03-23 00:00:00
2021-03-24 00:00:00


In [73]:
Agg_df#.dropna('all')#[~Agg_df.nan()]

,2603_長榮,2890_永豐金,2317_鴻海,2609_陽明,3006_晶豪科,8021_尖點,2303_聯電,2344_華邦電,3231_緯創,2328_廣宇,...,3520_華盈,4433_興采,5258_虹堡,6163_華電網,5272_笙科,2915_潤泰全,3558_神準,6605_帝寶,00878_國泰永續高股息,4908_前鼎
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,1937.0,2380.0,-271.0,-3225.0,-964.0,-665.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,-7153.0,1100.0,NaN,503.0,NaN,NaN,3076.0,2014.0,308.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-06,480.0,NaN,NaN,1322.0,NaN,NaN,-1977.0,NaN,NaN,1548.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-07,3085.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1625.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-08,-3005.0,233.0,NaN,702.0,NaN,NaN,-250.0,1536.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-160.0,NaN,NaN,NaN,NaN,NaN
2022-06-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-198.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
Agg_df.append(Agg_df.fillna(0).cumsum(), ignore_index=True)

,2603_長榮,2890_永豐金,2317_鴻海,2609_陽明,3006_晶豪科,8021_尖點,2303_聯電,2344_華邦電,3231_緯創,2328_廣宇,...,3520_華盈,4433_興采,5258_虹堡,6163_華電網,5272_笙科,2915_潤泰全,3558_神準,6605_帝寶,00878_國泰永續高股息,4908_前鼎
0,1937.0,2380.0,-271.0,-3225.0,-964.0,-665.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-7153.0,1100.0,NaN,503.0,NaN,NaN,3076.0,2014.0,308.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,480.0,NaN,NaN,1322.0,NaN,NaN,-1977.0,NaN,NaN,1548.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3085.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1625.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-3005.0,233.0,NaN,702.0,NaN,NaN,-250.0,1536.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-198.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347,121.0,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.0,182.0
348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-121.0


In [ ]:
Agg_df.fillna(0).cumsum().apply()